In [1]:
%matplotlib inline 
import os
import numpy as np
import tensorflow as tf
import pymongo as pm
import gridfs
import cPickle
import scipy.signal as signal
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, trange
from assignment_utils import *

In [19]:
collname = 'alexnet'
v_list = ['V0V3V6', 'V6']
target_layers = ['pool1', 'conv2', 'conv3', 'conv4', 'conv5', 'pool5', 'fc6', 'fc7']
step_list = [90000, 120000, 150000, 240000]

In [20]:
coll, db = p_get_coll(collname,'imagenet')
coll.distinct('exp_id')

[u'experiment_1',
 u'experiment_1_120000',
 u'experiment_1_90000',
 u'experiment_1_150000',
 u'experiment_1_240000']

In [24]:
def p_get_data_list(coll, collname, step, v, idx = -1):
    exp_id = 'experiment_1_%s_%s_%s'%(str(step), collname, v)
    def get_neural_validation_data(exp_id):
        q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates': {'$exists' : True}}
        val_steps = coll.find(q_val, projection = ['validation_results', 'validates', 'exp_id'])

        results = [val_steps[i] for i in range(val_steps.count())]
        for res in results:
            try:
                res['step'] = coll.find({'_id': res['validates']})[0]['step']
            except:
                res['step'] = -1
        return results

    validation_data = get_neural_validation_data(exp_id=exp_id)
    print idx
    data = validation_data[idx]['validation_results']['valid0']
    print validation_data[idx]['step'], v
    return data

In [25]:
data = {}
for iv in v_list:
    for istep in step_list:
        print iv, istep
        data[(iv, istep)] = p_get_data_list(coll, collname, istep, iv)

V0V3V6 90000
-1


IndexError: list index out of range

In [ ]:
def get_categorization_results(data, target_layers, step, category=None):
    """
    Plots the confusion matrix and the average classification accuracy for each layer.

    You will need to EDIT this section.
    """
    for i, layer in enumerate(target_layers):
        if category:
            k = 'within_categorization_%s' % layer                
            try:
                categorization_results = data[k][category]['result_summary']
            except:
                raise Exception(category + " is not a real category in data...")
        else:
            k = 'categorization_%s' % layer
            categorization_results = data[k]['result_summary']
        ### YOUR CODE HERE
        fig = plt.figure(figsize=(8, 8))
        m = fig.gca().matshow(np.array(categorization_results['cms']).mean(2))
        plt.xticks(range(8), categorization_results['labelset'])
        plt.yticks(range(8), categorization_results['labelset'])
        plt.colorbar(m)
        ave_acc = (np.mean(categorization_results['accbal']) - .5) * 2
        if category:
            plt.title("category: " + category + " , " + k + "\n: mean_accuracy = " + str(ave_acc) + ", iteration_step = " + str(step))
        else:
            plt.title(k + ": mean_accuracy = " + str(ave_acc) + "\n, iteration_step = " + str(step))

        m.set_cmap('jet')
        ax = plt.gca()
        ax.xaxis.tick_bottom()
        